In [36]:
import sqlite3
import pandas as pd

In [37]:


data = {
    'CHAVE': [3219283, 3219283, 3219283, 3219283, 3219283, 3219283, 3219283, 3219283, 3219283, 3219283, 3219283, 42205621, 42205621, 42205621, 42205621, 42205621, 42205621, 42205621, 7210639349, 7210639349, 7210639349, 7210639349],
    'DATAINICIO': ['20161107', '20161210', '20170107', '20170207', '20170307', '20170406', '20170506', '20190607', '20210108', '20230608', '20230708', '20210325', '20210424', '20220727', '20230324', '20230426', '20230523', '20230623', '20230309', '20230407', '20230509', '20230610'],
    'DATAFIM': ['20161209', '20170106', '20170206', '20170306', '20170405', '20170505', '20170607', '20190708', '20210208', '20230707', '20230808', '20210423', '20210526', '20220824', '20230425', '20230522', '20230622', '20230724', '20230406', '20230508', '20230609', '20230710'],
    'VSTELLE': [9219283, 9219283, 9219283, 9219283, 9219283, 9219283, 9219283, 9219283, 9219283, 9219283, 9219283, 92205621, 92205621, 92205621, 92205621, 92205621, 92205621, 92205621, 9210639349, 9210639349, 9210639349, 9210639349],
    'SPEBENE': [127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 220, 220, 220, 220],
    'ANLART': ['MFA', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT', 'FAT'],
    'ABLEINH': [43149021, 43149021, 43149021, 43149021, 43149021, 43149021, 43149021, 43149021, 43149021, 43149021, 43149021, 43149021, 43149021, 43149021, 43149021, 43149021, 43149021, 43149021, 78466, 78466, 78536, 78536],
    'TEMP_AREA': [139101, 139101, 139101, 139101, 139101, 139101, 139101, 139101, 139101, 139101, 139101, 139101, 139101, 139111, 139111, 139111, 139111, 139111, 133054, 133054, 133054, 133054],
    'ISTYPE': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3]
}

df = pd.DataFrame(data)

# Converter as colunas de data para o tipo datetime
df['DATAINICIO'] = pd.to_datetime(df['DATAINICIO'], format='%Y%m%d')
df['DATAFIM'] = pd.to_datetime(df['DATAFIM'], format='%Y%m%d')


conn = sqlite3.connect(':memory:')


df.to_sql('Faturas', conn, index=False, if_exists='replace')

# EXECUÇÃO DO SQL
query = '''
WITH BaseData AS (
    SELECT 
        CHAVE, 
        DATE(DATAINICIO) AS DATAINICIO, 
        DATE(DATAFIM) AS DATAFIM, 
        VSTELLE, 
        SPEBENE, 
        ANLART, 
        ABLEINH, 
        TEMP_AREA, 
        ISTYPE,
        LAG(DATE(DATAFIM)) OVER (PARTITION BY CHAVE ORDER BY DATE(DATAINICIO)) AS PREV_DATAFIM
    FROM 
        Faturas
),

NonConsolidated AS (
    SELECT 
        CHAVE, 
        DATAINICIO, 
        DATAFIM, 
        VSTELLE, 
        SPEBENE, 
        ANLART, 
        ABLEINH, 
        TEMP_AREA, 
        ISTYPE,
        PREV_DATAFIM
    FROM 
        BaseData
),

DummyFilled AS (
    SELECT 
        CHAVE, 
        DATAINICIO, 
        DATAFIM, 
        VSTELLE, 
        SPEBENE, 
        ANLART, 
        ABLEINH, 
        TEMP_AREA, 
        ISTYPE
    FROM 
        NonConsolidated
    UNION ALL
    SELECT 
        CHAVE, 
        DATE(PREV_DATAFIM, '+1 day') AS DATAINICIO, 
        DATE(DATAINICIO, '-1 day') AS DATAFIM,
        'DUMMY' AS VSTELLE, 
        'DUMMY' AS SPEBENE, 
        'DUMMY' AS ANLART, 
        'DUMMY' AS ABLEINH, 
        'DUMMY' AS TEMP_AREA, 
        'DUMMY' AS ISTYPE
    FROM 
        NonConsolidated
    WHERE 
        PREV_DATAFIM IS NOT NULL
        AND DATE(PREV_DATAFIM, '+1 day') < DATAINICIO
)

SELECT 
    CHAVE, 
    DATAINICIO, 
    DATAFIM, 
    VSTELLE, 
    SPEBENE, 
    ANLART, 
    ABLEINH, 
    TEMP_AREA, 
    ISTYPE
FROM 
    DummyFilled
ORDER BY 
    CHAVE, 
    DATAINICIO;
'''

result = pd.read_sql_query(query, conn)
result.head(23)
# RESULTADO


,CHAVE,DATAINICIO,DATAFIM,VSTELLE,SPEBENE,ANLART,ABLEINH,TEMP_AREA,ISTYPE
0,3219283,2016-11-07,2016-12-09,9219283,127,MFA,43149021,139101,1
1,3219283,2016-12-10,2017-01-06,9219283,127,FAT,43149021,139101,1
2,3219283,2017-01-07,2017-02-06,9219283,127,FAT,43149021,139101,1
3,3219283,2017-02-07,2017-03-06,9219283,127,FAT,43149021,139101,1
4,3219283,2017-03-07,2017-04-05,9219283,127,FAT,43149021,139101,1
5,3219283,2017-04-06,2017-05-05,9219283,127,FAT,43149021,139101,1
6,3219283,2017-05-06,2017-06-07,9219283,127,FAT,43149021,139101,1
7,3219283,2017-06-08,2019-06-06,DUMMY,DUMMY,DUMMY,DUMMY,DUMMY,DUMMY
8,3219283,2019-06-07,2019-07-08,9219283,127,FAT,43149021,139101,1
9,3219283,2019-07-09,2021-01-07,DUMMY,DUMMY,DUMMY,DUMMY,DUMMY,DUMMY


In [38]:
# Convertendo as colunas de data para o formato datetime e fazendo uma VERIFICAÇÃO
result['DATAINICIO'] = pd.to_datetime(result['DATAINICIO'], format='%Y%m%d', errors='coerce')
result['DATAFIM'] = pd.to_datetime(result['DATAFIM'], format='%Y%m%d', errors='coerce')

def verificar_gaps_e_sobreposicoes(df):
    erros = []
    for chave, group in df.groupby('CHAVE'):
        prev_datafim = None
        for idx, row in group.iterrows():
            if pd.notna(prev_datafim):
                if pd.notna(row['DATAINICIO']):
                    if row['DATAINICIO'] != prev_datafim + pd.Timedelta(days=1):
                        erros.append((row['CHAVE'], row['DATAINICIO'], prev_datafim + pd.Timedelta(days=1)))
                    if row['DATAINICIO'] <= prev_datafim:
                        erros.append((row['CHAVE'], row['DATAINICIO'], prev_datafim))
                else:
                    erros.append((row['CHAVE'], row['DATAINICIO'], prev_datafim + pd.Timedelta(days=1)))
            prev_datafim = row['DATAFIM']
    return erros

# Executando a verificação
erros = verificar_gaps_e_sobreposicoes(result)

# Exibindo os erros encontrados
if erros:
    print("Foram encontrados erros nos períodos de datas:")
    for erro in erros:
        print(f"CHAVE: {erro[0]}, DATAINICIO: {erro[1]}, DATAFIM esperado: {erro[2]}")
else:
    print("Nenhum erro encontrado nos períodos de datas.")

Nenhum erro encontrado nos períodos de datas.
